In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import basedosdados as bd
import re
import json
import string
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import os

In [ ]:
municipios = pd.read_csv("csv/municipios.csv")
zonas_eleitorais = pd.read_csv("csv/zona_eleitoral_2.csv", error_bad_lines=False, delimiter=',')

zonas_eleitorais = zonas_eleitorais[zonas_eleitorais['uf'].eq('RJ') & zonas_eleitorais['nome_municipio'].eq('RIO DE JANEIRO')]
zonas_eleitorais['id'].replace('(19-0*)','',regex=True, inplace = True)

with open('csv/bairros-geojson.json', encoding="utf8") as geojson:
    bairros = json.load(geojson)  

In [ ]:
def dataAlreadyBeenGot(csvName):
    if os.stat(csvName).st_size == 0:
        return False
    else:
        return True

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    queryResultadosPartidoMunicipio = """
        SELECT MZ.ano, MZ.zona, MZ.sigla_partido, MZ.cargo, MZ.turno, MZ.votos_nominais, MZ.votos_nao_nominais FROM `basedosdados.br_tse_eleicoes.resultados_partido_municipio_zona` AS MZ
        WHERE MZ.id_municipio_tse={} AND MZ.sigla_uf='RJ'
        ORDER BY MZ.ano ASC""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0])

    dfResultadosPartidoMunicipio = bd.read_sql(queryResultadosPartidoMunicipio, billing_project_id='capacitacaoufrjanalytica')
    dfResultadosPartidoMunicipio.to_csv("csv/resultado_eleicoes.csv", index=False)
else:
    dfResultadosPartidoMunicipio = pd.read_csv("csv/resultado_eleicoes.csv")

In [ ]:
def neighhboorhoodCoordinateIsInside(latitude, longitude):
    for e in bairros['features']:
        if (e['geometry']['type'] != 'MultiPolygon'):
            vertices = e['geometry']['coordinates'][0]
        else:
            vertices = e['geometry']['coordinates'][0][0]
        wn = 0;
        i = 0
        j = len(vertices) - 1
        while (i < len(vertices)):
            v = vertices[j];
            v1 = vertices[i];
            if ( v[0] <= longitude ) :
                if ( longitude < v1[0] ):  
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) > 0 ):
                        wn += 1;
            else:
                if( longitude >= v1[0] ):
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) < 0 ):
                        wn -= 1;
            j = i
            i += 1
    
        if (wn != 0):
            return e['properties']['NOME']
        
def returnLat(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['latitude'].values[0] 
def returnLon(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['longitude'].values[0] 
def returnBairro(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['bairro'].values[0]

In [ ]:
dfResultadosPartidoMunicipio['bairro'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnBairro(x))

In [ ]:
dfResultadosPartidoMunicipio['latitude'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnLat(x))
dfResultadosPartidoMunicipio['longitude'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnLon(x))

In [ ]:
auxDf = dfResultadosPartidoMunicipio[~(dfResultadosPartidoMunicipio['cargo'].eq('presidente') | dfResultadosPartidoMunicipio['cargo'].eq('governador') | dfResultadosPartidoMunicipio['cargo'].eq('prefeito'))]

def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_votosArray = []
    partidosArray = []
    latitudeArray = []
    longitudeArray = []
    for bairro, frame in auxDf[auxDf['ano'].eq(ano)].groupby('bairro'):
        n_votos = np.max(frame['votos_nominais'])
        partido = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['sigla_partido'].values[0]
        latitude = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['latitude'].values[0]
        longitude = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['longitude'].values[0]
        n_votosArray.append(n_votos)
        partidosArray.append(partido)
        latitudeArray.append(latitude)
        longitudeArray.append(longitude)
        bairrosArray.append(neighhboorhoodCoordinateIsInside(latitude, longitude))

    dfAgrupadoResultadosPartidoMunicipio = pd.DataFrame([bairrosArray, n_votosArray, partidosArray], 
                              index=['bairro', 'votos', 'partido']).T
    
    dfAgrupadoResultadosPartidoMunicipio['ano'] = [ano]*len(dfAgrupadoResultadosPartidoMunicipio)
    
    return dfAgrupadoResultadosPartidoMunicipio

dfAgrupadoResultadosPartidoMunicipioPorAno = pd.DataFrame()
for a in dfResultadosPartidoMunicipio['ano'].unique():
    dfAgrupadoResultadosPartidoMunicipioPorAno = dfAgrupadoResultadosPartidoMunicipioPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoResultadosPartidoMunicipioPorAno.reset_index(drop=True, inplace=True)

In [ ]:
ano = 2016
fig = px.choropleth_mapbox(dfAgrupadoResultadosPartidoMunicipioPorAno[dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.DataFrame()
    for ano in dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].unique():
        queryFiliacaoPartidaria = """
            SELECT COUNT(*) AS n_filiados, FP.zona, FP.sigla_partido FROM `basedosdados.br_tse_filiacao_partidaria.filiacao_partidaria` AS FP
            WHERE FP.id_municipio_tse={} AND (FP.data_desfiliacao > '{}-12-30' OR (FP.situacao_registro = 'regular' AND FP.data_filiacao < '{}-01-01'))
            GROUP BY FP.zona, FP.sigla_partido
            ORDER BY FP.zona""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0], ano, ano)

        dfResultadoQuery = bd.read_sql(queryFiliacaoPartidaria, billing_project_id='capacitacaoufrjanalytica')
        dfResultadoQuery['ano'] = [ano]*len(dfResultadoQuery)
        dfFiliacaoPartidaria = dfFiliacaoPartidaria.append(dfResultadoQuery)
    dfFiliacaoPartidaria.to_csv("csv/filiados.csv", index=False)
else:
    dfFiliacaoPartidaria = pd.read_csv("csv/filiados.csv")


In [ ]:
dfFiliacaoPartidaria['bairro'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnBairro(x))

In [ ]:
dfFiliacaoPartidaria['latitude'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnLat(x))
dfFiliacaoPartidaria['longitude'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnLon(x))

In [ ]:
def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_filiadosArray = []
    partidosArray = []
    latitudeArray = []
    longitudeArray = []
    for bairro, frame in dfFiliacaoPartidaria[dfFiliacaoPartidaria['ano'].eq(ano)].groupby('bairro'):
        n_filiados = np.max(frame['n_filiados'])
        partido = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['sigla_partido'].values[0]
        latitude = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['latitude'].values[0]
        longitude = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['longitude'].values[0]

        n_filiadosArray.append(n_filiados)
        partidosArray.append(partido)
        latitudeArray.append(latitude)
        longitudeArray.append(longitude)
        bairrosArray.append(neighhboorhoodCoordinateIsInside(latitude, longitude))
        
    
    dfAgrupadoFiliacaoPartidaria = pd.DataFrame([bairrosArray, n_filiadosArray, partidosArray], 
                              index=['bairro', 'filiados', 'partido']).T
    
    dfAgrupadoFiliacaoPartidaria['ano'] = [ano]*len(dfAgrupadoFiliacaoPartidaria)
    
    return dfAgrupadoFiliacaoPartidaria

dfAgrupadoFiliacaoPartidariaPorAno = pd.DataFrame()
for a in dfFiliacaoPartidaria['ano'].unique():
    dfAgrupadoFiliacaoPartidariaPorAno = dfAgrupadoFiliacaoPartidariaPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoFiliacaoPartidariaPorAno.reset_index(drop=True, inplace=True)                     

In [ ]:
ano = 2012
fig = px.choropleth_mapbox(dfAgrupadoFiliacaoPartidariaPorAno[dfAgrupadoFiliacaoPartidariaPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['filiados'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
dfResultadosPartidoMunicipio = dfResultadosPartidoMunicipio.replace({'PC do B': 'PCDOB'})
votosArray = []
anosArray = []
partidosArray = []
for group, frame in dfResultadosPartidoMunicipio.set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)):
    n_votos = np.sum(frame['votos_nominais']) + np.sum(frame['votos_nao_nominais']) 
    votosArray.append(n_votos)
    anosArray.append(group[0])
    partidosArray.append(group[1])

dfVotosAgrupados = pd.DataFrame([votosArray, anosArray, partidosArray],
                               index=['votos', 'ano', 'partido']).T
dfVotosAgrupados

In [ ]:
filiadosArray = []
anosArray = []
partidosArray = []
for group, frame in dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)):
    n_filiados = np.sum(frame['n_filiados'])
    filiadosArray.append(n_filiados)
    anosArray.append(group[0])
    partidosArray.append(group[1])

dfFiliadosAgrupados = pd.DataFrame([filiadosArray, anosArray, partidosArray],
                               index=['filiados', 'ano', 'partido']).T
dfFiliadosAgrupados

In [ ]:
new_df = pd.merge(dfFiliadosAgrupados, dfVotosAgrupados,  how='inner', left_on=['ano', 'partido'], right_on = ['ano', 'partido'])

In [ ]:
new_df['filiados'] = new_df['filiados'].apply(lambda x: int(x)) 
new_df['votos'] = new_df['votos'].apply(lambda x: int(x)) 
new_df.corr()

In [ ]:
fig = px.scatter(new_df, x="filiados", y="votos", color='partido')
fig.show()